In [36]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

df = spark.sql("SELECT * FROM Bing_Lakehouse.tbl_latest_news")
display(df)

StatementMeta(, 8421fb41-11d5-4249-bbb3-a0bb9c9258db, 38, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 288d6f23-293b-4243-af97-dc1efe65a232)

In [37]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 8421fb41-11d5-4249-bbb3-a0bb9c9258db, 39, Finished, Available, Finished)

In [38]:
#import the model and configure input output column
model = (
    AnalyzeText()
    .setKind("SentimentAnalysis")
    .setTextCol("description")
    .setOutputCol("response")
    .setErrorCol("error")
)

StatementMeta(, 8421fb41-11d5-4249-bbb3-a0bb9c9258db, 40, Finished, Available, Finished)

In [39]:
#apply the model to df
result = model.transform(df)

StatementMeta(, 8421fb41-11d5-4249-bbb3-a0bb9c9258db, 41, Finished, Available, Finished)

In [40]:
display(result)

StatementMeta(, 8421fb41-11d5-4249-bbb3-a0bb9c9258db, 42, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5facfe41-bda3-4788-b4c8-f2ef3cd76529)

In [41]:
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))



StatementMeta(, 8421fb41-11d5-4249-bbb3-a0bb9c9258db, 43, Finished, Available, Finished)

In [42]:
display(sentiment_df)

StatementMeta(, 8421fb41-11d5-4249-bbb3-a0bb9c9258db, 44, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 97cbe0c9-db0e-495c-8c1b-ce9aad51621b)

In [43]:
sentiment_df_final = sentiment_df.drop("response", "error")

StatementMeta(, 8421fb41-11d5-4249-bbb3-a0bb9c9258db, 45, Finished, Available, Finished)

In [44]:
display(sentiment_df_final)

StatementMeta(, 8421fb41-11d5-4249-bbb3-a0bb9c9258db, 46, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0702e2c9-e8eb-435e-ad8a-f54d6528cb0b)

In [56]:
from pyspark.sql.functions import col, to_date
sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, 8421fb41-11d5-4249-bbb3-a0bb9c9258db, 58, Finished, Available, Finished)

In [45]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'Bing_Lakehouse.tbl_sentiment_analysis'

    sentiment_df_final.write.format('delta').saveAsTable(table_name)

except AnalysisException:
    print("Table already exists")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_sentiment_df_final source_view
                    
                    ON source_view.url = target_table.url
                    
                    WHEN MATCHED AND 
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished 
                    
                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *
                """)


StatementMeta(, 8421fb41-11d5-4249-bbb3-a0bb9c9258db, 47, Finished, Available, Finished)

Table already exists
